In [1]:
import os
import sys
root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)
!{sys.executable} -m pip install numpy pandas intervaltree matplotlib



You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
from strategy.data import RawData
from strategy.primitives import Pool, Token, Fee
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)
raw = RawData.from_pool(pool)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from strategy.primitives import Pool, Frequency
from strategy.data import PoolData

p = PoolData.from_raw_data(raw, Frequency.DAY)
display(p.data.tail())
p.plot()


In [ ]:
from strategy.portfolio import Position
from strategy.backtest import AbstractStrategy, Backtest
from strategy.data import PoolData
from datetime import datetime
from typing import Callable

class RebalanceStrategy(AbstractStrategy):        
    def rebalance(
        self,
        t: datetime,
        c: float,
        vol: float,
        l: Callable[[float], float],
        pool_data: PoolData,
    ) -> bool:
        if not self.portfolio.position("main"):
            self.portfolio.add_position(Position(id="main", a = c / 1.0001 ** 60, b = c * 1.0001 ** 60))
            pos = self.portfolio.position("main")
            pos.deposit(c, 1)
        else:
            pos = self.portfolio.position("main")
            pos.set_a(c / 1.0001 ** 60, c)
            pos.set_b(c * 1.0001 ** 60, c)

        return True
backtest = Backtest(RebalanceStrategy())
# data = p["2021-06-20":"2021-06-30"]
data = p
backtest.run(data, 0.001)
backtest.plot(20, 50)